In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Read train and test test data
train_data = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
test_data = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')

In [ ]:
# view data samples for train data
train_data.sample(10)

* SalePrice is our target variable.we have to predict best Sale price of house

### Descriptive Analysis

In [ ]:
# Check the information of datasets
print('*' * 20 ,'Traning Data ',"*"*20)
print(train_data.info())
print('*' * 20 ,'Test Data ',"*"*20)
print(test_data.info())

### Training Data
* There are total 1460 observations with 81 columns/variables/features.
* There are both numerical and categorical data.

In [ ]:
pd.isnull(train_data).sum()

### Missing Values in particular column/Feature/Attribute

In [ ]:
for i in train_data.columns:
    print(i,'-->',train_data[i].isnull().sum())

In [ ]:
sns.heatmap(train_data.isnull(),yticklabels=False, cmap='plasma')

In [ ]:
# columns which have missing values
train_data.columns[train_data.isnull().any()]

In [ ]:
missing_data_list = train_data.columns[train_data.isnull().any()]

### List of columns which has some mising data

In [ ]:
list(missing_data_list)

In [ ]:
for i in missing_data_list:
    print(i,'--->',round(train_data[i].isna().sum()/len(train_data) *100,2),'%' )

### Test Data
* There are total 1459 observations with 80 columns/variables/features.
* There are both numerical and categorical data.

In [ ]:
pd.isnull(test_data).sum()

In [ ]:
sns.heatmap(test_data.isnull(),yticklabels=False, cmap='plasma')

In [ ]:
# columns which contains missing values in test data
test_data.columns[test_data.isnull().any()]

In [ ]:
# percentage of of missing values
missing_col = list(test_data.columns[test_data.isnull().any()])
for i in missing_col:
    print(i,'-->',round(test_data[i].isna().sum()/len(train_data) *100,2),'%')

In [ ]:
# Descriptive mesure of train_data
train_data.describe(include='all')

In [ ]:
train_data.describe()

In [ ]:
# for Test_data 
test_data.describe(include ='all')

In [ ]:
test_data.describe()

### Data Preprocessing
* <b>Treatement of missing Values</b>

#### Dropping columns with missing values >= 20%

In [ ]:
columns = []
for i in train_data.columns:
    if train_data[i].isna().sum()/len(train_data)*100 >=10:
        columns.append(i)
print(columns)# List of columns which has >=10% missing data       

In [ ]:
# Droping columns in train data
train_data = train_data.drop(columns=columns,axis = 1)
train_data.sample(5)

* 5 Columns dropped from train and test data

* check categorical and numerical columns in dataset

In [ ]:
train_data.info()

In [ ]:
feature_data = train_data.drop(columns=['SalePrice'])
target_data = train_data.SalePrice

In [ ]:
feature_data.info()

In [ ]:
# select int data in to one variable
float_int_data = feature_data.select_dtypes(include = ['int','float'])

In [ ]:
# select object data into one variable
cat_data = feature_data.select_dtypes(include = ['object'])

In [ ]:
from sklearn.pipeline import make_pipeline

In [ ]:
from sklearn.preprocessing import OrdinalEncoder,MinMaxScaler

In [ ]:
from sklearn.impute import SimpleImputer

In [ ]:
# creating sub pipelines
float_int_pipeline = make_pipeline(SimpleImputer(strategy='median'),MinMaxScaler())
cat_pipeline = make_pipeline(SimpleImputer(strategy='most_frequent'),OrdinalEncoder())

In [ ]:
from sklearn.compose import make_column_transformer

In [ ]:
preprocessor = make_column_transformer(
               (cat_pipeline,cat_data.columns),
               (float_int_pipeline,float_int_data.columns)
               )

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier

In [ ]:
pipeline = make_pipeline(preprocessor, LinearRegression())

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
trainX, testX, trainY, testY = train_test_split(feature_data, target_data)

In [ ]:
pipeline.fit(feature_data, target_data)

In [ ]:
pipeline.score(testX,testY)

In [ ]:
pipeline = make_pipeline(preprocessor, RandomForestClassifier())

In [ ]:
pipeline.fit(feature_data, target_data)

In [ ]:
pipeline.score(testX, testY)

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
y_pred = pipeline.predict(testX)

In [ ]:
mean_squared_error(testY, y_pred)

In [ ]:
y_pred = pipeline.predict(test_data)
submission = pd.DataFrame({'Id': test_data.index,'SalePrice': y_pred})

In [ ]:
submission.to_csv("house_prices_submission.csv", index=False)